# Initialization

In [1]:
import imp
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import numpy as np
#from data_parser import DataParser
from pull_data import Pull
import os
from sklearn.metrics import confusion_matrix
from prettytable import PrettyTable
from statistics import mean
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

#from scikit_IsolatedForest import RandomForest
from sklearn.ensemble import RandomForestClassifier
#from scikit_LOFNovelty import AdaBoost
from sklearn.ensemble import AdaBoostClassifier
#from scikit_OneClassSVM import GradientBoosting
from sklearn.ensemble import GradientBoostingClassifier

In [107]:
DATA_CLASS = {
    # CESNET DATASET
    "IKEA_APP" : "dev-annotated-datasets/ikea-app/train",
    #"IKEA_HOMEKIT_CLEAR" : "dev-annotated-datasets/ikea-app/anomaly",
    #"IKEA_HOMEKIT" : "dev-annotated-datasets/ikea-homekit/train",
    "IP_CAM" : "dev-annotated-datasets/ipcam/train",
    #"IP_CAM_MISCONFIG" : "dev-annotated-datasets/ipcam/anomaly_cam",
    "NORMAL_USER" : "dev-annotated-datasets/normal-user/train",
    "VOICE_ASSISTANT" : "dev-annotated-datasets/voice-assistant/train",
    
    # UNSW DATASET
    ## HUBS
    "AMAZON_ECHO" : "../data-sets/unsw-traces-device/Amazon-Echo",
    "SMART_THINGS" : "../data-sets/unsw-traces-device/Smart-Things",
    ## CAMERAS
    "NETAMO_WELCOME" : "../data-sets/unsw-traces-device/Netatmo-Welcome",
    #"TP-Link-Day-Night-Cloud-Camera" : "../data-sets/unsw-traces-device/TP-Link-Day-Night-Cloud-Camera",
    "Samsung-Smart-Cam" : "../data-sets/unsw-traces-device/Samsung-Smart-Cam",
    "INSTEON_CAM" : "../data-sets/unsw-traces-device/Insteon-Camera",
    #"DROP_CAM" : "../data-sets/unsw-traces-device/Dropcam",
    "WITHINGS_SMART_BABY_MONITOR" : "../data-sets/unsw-traces-device/Withings-Smart-Baby-Monitor",
    ## SWITCHES AND TRIGGERS
    "BELKIN_WEMO_SWITCH" : "../data-sets/unsw-traces-device/Belkin-Wemo-Switch",
    "TP-Link-Smart-Plug" : "../data-sets/unsw-traces-device/TP-Link-Smart-Plug",
    #"iHome" : "../data-sets/unsw-traces-device/iHome",
    "BELKIN_WEMO_MOTION_SENSOR" : "../data-sets/unsw-traces-device/Belkin-Wemo-Motion-Sensor",
    ## AIR QUALITY SENSORS
    #"NEST-Protect-Smoke-Alarm" : "../data-sets/unsw-traces-device/NEST-Protect-Smoke-Alarm",
    "Netatmo-Weather-Station" : "../data-sets/unsw-traces-device/Netatmo-Weather-Station",
    ## HEATLTHCARE DEVICE
    #"Withings-Smart-Scale" : "../data-sets/unsw-traces-device/Withings-Smart-Scale",
    #"Blipcare-Blood-Pressure-Meter" : "../data-sets/unsw-traces-device/Blipcare-Blood-Pressure-Meter",
    "Withings-Aura-Smart-Sleep-Sensor" : "../data-sets/unsw-traces-device/Withings-Aura-Smart-Sleep-Sensor",
    ## LIGHT BULBS
    "Light-Bulbs-LiFX-Smart-Bulb" : "../data-sets/unsw-traces-device/Light-Bulbs-LiFX-Smart-Bulb",
    ## ELECTRONIC
    #"Triby-Speaker" : "../data-sets/unsw-traces-device/Triby-Speaker",
    #"PIX-STAR-Photo-Frame" : "../data-sets/unsw-traces-device/PIX-STAR-Photo-Frame",
    "HP-Printer" : "../data-sets/unsw-traces-device/HP-Printer",
    ## NON-IOT
    "Laptop" : "../data-sets/unsw-traces-device/Laptop",
    "ANDROID_PHONE" : "../data-sets/unsw-traces-device/Android-Phone",
    "Samsung-Galaxy-Tab" : "../data-sets/unsw-traces-device/Samsung-Galaxy-Tab",
    #"IPhone" : "../data-sets/unsw-traces-device/IPhone",
    
    # CTU13 BOTNET ATTACKS DATASET
    #"BOTNET_SOGOU" : "../data-sets/botnet/sogou",
    #"BOTNET_RBOT" : "../data-sets/botnet/rbot",
     "BOTNET_NERIS" : "../data-sets/botnet/neris"
}

# Function Definitions

In [3]:
def runModel(models):
    for key, model in models.items():
        print("### Model Name:",key," ###")
        kf = KFold(5, True)
        iteration_cnt = 0
        for train_index, test_index in kf.split(c_data,c_target):
            iteration_cnt += 1
            #Train
            model.fit(c_data[train_index],c_target[train_index])
            #Evaluate 
            p_class = model.predict(c_data[test_index])
            y_pred_valid = model.predict_proba(c_data[test_index])
            #print(p_class)

# Pull Datasets

# MODELS

In [4]:
MODELS = {}
MODELS["RandomForest"] = {}
MODELS["AdaBoost"] = {}
MODELS["GradientBoosting"] = {}
rng = np.random.RandomState(12345)

MODELS["RandomForest"]["RF1"] = RandomForestClassifier(random_state=rng,n_estimators=10)
MODELS["AB1"] = AdaBoostClassifier(random_state=rng)
MODELS["GradientBoosting"] = GradientBoostingClassifier(random_state=rng)


In [5]:
#runModel(MODELS["RandomForest"])

# Evaluation

In [110]:
UPPER_LIMIT = 400
ANOMALY = ["BOTNET_NERIS"]
class_index = 1
c_data = None
c_target = None
c_anomaly_data = None
c_anomaly_target = None

for data_cl in DATA_CLASS: 
    
    if data_cl in ANOMALY:
        a = Pull(DATA_CLASS[data_cl],999)
        if len(a.data) < UPPER_LIMIT:
            max_limit = len(a.data)
        else:
            max_limit = UPPER_LIMIT
        idx = np.random.choice(len(a.data), max_limit, replace=False)
        
        if c_anomaly_data is None:
            #c_anomaly_data = np.array(a.data[0:UPPER_LIMIT])
            #c_anomaly_target = np.array(a.labels[0:UPPER_LIMIT])
            c_anomaly_data = np.array(a.data)[idx]
            c_anomaly_target = np.array(a.labels)[idx]
        else:
            #c_anomaly_data = np.concatenate((c_anomaly_data,a.data[0:UPPER_LIMIT]))
            #c_anomaly_target = np.concatenate((c_anomaly_target,a.labels[0:UPPER_LIMIT]),axis=None)
            c_anomaly_data = np.concatenate((c_anomaly_data,np.array(a.data)[idx]))
            c_anomaly_target = np.concatenate((c_anomaly_target,np.array(a.labels)[idx]),axis=None)
        continue
        
    a = Pull(DATA_CLASS[data_cl],class_index)
    if len(a.data) < UPPER_LIMIT:
        max_limit = len(a.data)
    else:
        max_limit = UPPER_LIMIT
    idx = np.random.choice(len(a.data), max_limit, replace=False)
        
    print("Loading",data_cl," dataset")
    if c_data is None:
        #c_data = np.array(a.data[0:UPPER_LIMIT])
        #c_target = np.array(a.labels[0:UPPER_LIMIT])
        c_data = np.array(a.data)[idx]
        c_target = np.array(a.labels)[idx]
    else:
        #c_data = np.concatenate((c_data,a.data[0:UPPER_LIMIT]))
        #c_target = np.concatenate((c_target,a.labels[0:UPPER_LIMIT]),axis=None)
        c_data = np.concatenate((c_data,np.array(a.data)[idx]))
        c_target = np.concatenate((c_target,np.array(a.labels)[idx]),axis=None)
        
    #print(len(a.data),len(a.labels))
    #print(len(c_data),len(c_target))
    #print(type(a.data),type(c_data))
    #print(c_data[0])
    #print("Flows:",a.flow_cnt," Errors:",a.errors)
    class_index += 1
    #input("Press enter")

    #runModel(MODELS["RandomForest"])
    
    

Loading IKEA_APP  dataset
Loading IP_CAM  dataset
Loading NORMAL_USER  dataset
Loading VOICE_ASSISTANT  dataset
Loading AMAZON_ECHO  dataset
Loading SMART_THINGS  dataset
Loading NETAMO_WELCOME  dataset
Loading Samsung-Smart-Cam  dataset
Loading INSTEON_CAM  dataset
Loading WITHINGS_SMART_BABY_MONITOR  dataset
Loading BELKIN_WEMO_SWITCH  dataset
Loading TP-Link-Smart-Plug  dataset
Loading BELKIN_WEMO_MOTION_SENSOR  dataset
Loading Netatmo-Weather-Station  dataset
Loading Withings-Aura-Smart-Sleep-Sensor  dataset
Loading Light-Bulbs-LiFX-Smart-Bulb  dataset
Loading HP-Printer  dataset
Loading Laptop  dataset
Loading ANDROID_PHONE  dataset
Loading Samsung-Galaxy-Tab  dataset


In [7]:
print(len(c_data),len(c_target))

1443 1443


# Single Models

## Random Forest

In [64]:
def createResultDict(no_classes):
    tmp_struct = {}
    for i in range(no_classes):
        tmp_struct[i+1] = {}
        for j in range(no_classes):
            tmp_struct[i+1][j+1] = 0
    return tmp_struct
        

In [65]:
createResultDict(5)

{1: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0},
 2: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0},
 3: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0},
 4: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0},
 5: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0}}

In [113]:
rng = np.random.RandomState(12345)
model = RandomForestClassifier(random_state=rng,n_estimators=10)

kf = KFold(5, True)
iteration_cnt = 0

result_struct = createResultDict(20)

for train_index, test_index in kf.split(c_data,c_target):
    tmp_result_struct = {}#{1:{1:0,2:0,3:0,4:0},2:{1:0,2:0,3:0,4:0},3:{1:0,2:0,3:0,4:0},4:{1:0,2:0,3:0,4:0}}
    iteration_cnt += 1
    #Train
    model.fit(c_data[train_index],c_target[train_index])
    #Evaluate 
    y_class = model.predict(c_data[test_index])
    y_pred = model.predict_proba(c_data[test_index])
    
    for i in range(len(test_index)):
       # if c_target[i] == 1:
        try:
            tmp_result_struct[c_target[test_index[i]]][y_class[i]] += 1
        except Exception as e:
            try:
                tmp_result_struct[c_target[test_index[i]]][y_class[i]] = 1
            except Exception as e3:
                tmp_result_struct[c_target[test_index[i]]] = {}
                tmp_result_struct[c_target[test_index[i]]][y_class[i]] = 1
                

    for t_class,t_value in tmp_result_struct.items():
        p_sum = 0
        for p_class,p_value in t_value.items():
            p_sum += p_value
        for p_class,p_value in t_value.items():
            if iteration_cnt == 1:
                result_struct[t_class][p_class] = (p_value/p_sum)
            else:
                result_struct[t_class][p_class] = (result_struct[t_class][p_class]+(p_value/p_sum))/2

for key,val in result_struct.items():
    print(key,":",val)
#print(result_struct)
  
    #for i in range(len(y_class)):
    #    if y_class[i] != c_target[i]:
    #        print("Misclassification",y_pred[i],y_class[i],c_target[i])
    #        input("Press enter")
    #    print(y_pred[i],y_class[i],c_target[i])
    #    input("Press enter")
   # print(metrics.accuracy_score(y_class,c_target[test_index]))
   # print(classification_report(y_class,c_target[test_index],output_dict=False))

print(cross_val_score(model, c_data, c_target, cv=5))
print(classification_report(y_class,c_target[test_index],output_dict=False))
print(metrics.accuracy_score(y_class,c_target[test_index]))
print("===================")
y_class = model.predict(c_anomaly_data)
y_pred = model.predict_proba(c_anomaly_data)
#print(y_pred)
print(metrics.accuracy_score(y_class,[1]*len(c_anomaly_data.data)))
print(metrics.accuracy_score(y_class,[2]*len(c_anomaly_data.data)))
print(metrics.accuracy_score(y_class,[3]*len(c_anomaly_data.data)))
print(metrics.accuracy_score(y_class,[4]*len(c_anomaly_data.data)))
#print(classification_report(y_class,c_target[test_index],output_dict=False))


1 : {1: 0.980242713606989, 2: 0, 3: 0.006666666666666667, 4: 0.02197802197802198, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0.012073452862926548, 15: 0, 16: 0, 17: 0, 18: 0.0136986301369863, 19: 0.012161172161172162, 20: 0}
2 : {1: 0.029553699233361104, 2: 0.5968786466740651, 3: 0.008064516129032258, 4: 0.028099798387096774, 5: 0.35296159530481136, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0.015873015873015872, 17: 0.017239583333333336, 18: 0.0078125, 19: 0, 20: 0.015625}
3 : {1: 0.026384711779448622, 2: 0, 3: 0.8704966460268317, 4: 0.013157894736842105, 5: 0.012537593984962406, 6: 0, 7: 0.012515664160401003, 8: 0, 9: 0.005952380952380952, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0.0058823529411764705, 18: 0.02161626492702344, 19: 0.04147759103641457, 20: 0.018781328320802003}
4 : {1: 0.03914975560792529, 2: 0.03822626632917285, 3: 0.01694246239562422, 4: 0.8138325732448866, 5: 0.05694748116144424, 6: 0.018518518518518517, 7: 

## AdaBoost

## GradientBoosting